In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys; sys.path.insert(0, '..')

In [ ]:
from pathlib import Path

import fiona
import geopandas as gpd
import numpy as np
import osmnx as ox
import pandas as pd
from shapely.geometry import LineString, Point
from tqdm import tqdm
import altair as alt

In [ ]:
from main import prepare_data_for_place, OUTPUT_COLUMNS
from src.route import (
    get_route_gdf,
    compute_routes_from_census_blocks_to_school,
    compute_routes_from_census_blocks_to_all_schools
)

In [ ]:
def is_connected(g):
    import networkx as nx
    return nx.is_connected(g.to_undirected())

## Prepare out dir

In [ ]:
import os
import shutil

OUT_PATH = Path("../data/out/notebook/")

# Delete the directory if it exists
if OUT_PATH.exists():
    shutil.rmtree(OUT_PATH)

# Recreate the directory
OUT_PATH.mkdir(parents=True, exist_ok=True)

## Load bike network

In [ ]:
place = "Somerville, MA, USA"
nodes, edges = prepare_data_for_place(place)

In [ ]:
edges = edges[OUTPUT_COLUMNS]

In [ ]:
G = ox.graph_from_gdfs(nodes, edges)

In [ ]:
edges.sample(3)

In [ ]:
is_connected(G)

## Load Schools

In [ ]:
school_gdb_path = "../data/raw/SafeRoutesGISLayers.gdb.zip"

In [ ]:
layers = fiona.listlayers(school_gdb_path)
layers

In [ ]:
# read school data
schools_gdf = gpd.read_file(school_gdb_path, layer='PublicSchools')

# save schools polygons
schools_gdf.to_file((OUT_PATH / "schools_poly.gpkg"), driver="GPKG")

# make geom col into centroids
schools_gdf['geometry'] = schools_gdf.centroid

# save schools polygons
schools_gdf.to_file((OUT_PATH / "schools_centroid.gpkg"), driver="GPKG")

In [ ]:
schools_gdf.head(3)

## Load census blocks

In [ ]:
# read census blocks
census_blocks = gpd.read_file("../data/raw/Census_2020_Blocks.zip")

# filter by TOWN attribute
somerville_census_blocks = census_blocks[census_blocks['TOWN'] == "SOMERVILLE"].copy()

# reset index
somerville_census_blocks = somerville_census_blocks.reset_index(drop=True)

# get a sample
half_n_census_blocks = len(somerville_census_blocks) // 4
somerville_census_sample = somerville_census_blocks.sample(half_n_census_blocks)

# save polygon version
somerville_census_blocks.to_file((OUT_PATH / "somer_blocks_poly.gpkg"), driver="GPKG")
somerville_census_sample.to_file((OUT_PATH / "somer_sample_poly.gpkg"), driver="GPKG")

# convert geometry to centroid
somerville_census_blocks['geometry'] = somerville_census_blocks.centroid
somerville_census_sample['geometry'] = somerville_census_sample.centroid

# save centroid version
somerville_census_blocks.to_file((OUT_PATH / "somer_blocks_centroid.gpkg"), driver="GPKG")
somerville_census_sample.to_file((OUT_PATH / "somer_sample_centroid.gpkg"), driver="GPKG")

In [ ]:
somerville_census_blocks.head(3)

## Make sure everything has same crs

- EPSG:26986 =  NAD83 / Massachusetts Mainland Meters
- EPSG:4326 = WGS 84 / web

In [ ]:
def crs_first_line(gdf):
    return str(gdf.crs).splitlines()[0]

In [ ]:
print("somerville_census_blocks:", crs_first_line(somerville_census_blocks))
print("somerville_census_sample:", crs_first_line(somerville_census_sample))
print("schools_gdf             :", crs_first_line(schools_gdf))
print("edges                   :", crs_first_line(edges))
print("nodes                   :", crs_first_line(nodes))

In [ ]:
# use this one
use_crs = edges.crs

# make them match
somerville_census_blocks = somerville_census_blocks.to_crs(use_crs)
somerville_census_sample = somerville_census_sample.to_crs(use_crs)
schools_gdf = schools_gdf.to_crs(use_crs)
nodes = nodes.to_crs(use_crs)
edges = edges.to_crs(use_crs)

In [ ]:
print("somerville_census_blocks:", crs_first_line(somerville_census_blocks))
print("somerville_census_sample:", crs_first_line(somerville_census_sample))
print("schools_gdf             :", crs_first_line(schools_gdf))
print("edges                   :", crs_first_line(edges))
print("nodes                   :", crs_first_line(nodes))

## Routing

In [ ]:
G = ox.graph_from_gdfs(nodes, edges)
is_connected(G)

In [ ]:
# pick a school
dest_point = schools_gdf.loc[0, 'geometry']

In [ ]:
# pick a census centroid
orig_point = somerville_census_sample.iloc[0]['geometry']

In [ ]:
# route based on composite score
route_gdf = get_route_gdf(G, orig_point, dest_point)
route_gdf

In [ ]:
# route based on length
route_gdf = get_route_gdf(G, orig_point, dest_point, weight="length")
route_gdf

## Route loop

In [ ]:
# loop over all census blocks, computing routes to one school
combined_gdf, errors = compute_routes_from_census_blocks_to_school(
    G, somerville_census_blocks,
    schools_gdf.loc[0],
    weight="composite_score"
)

In [ ]:
print("\n".join(errors))
print("\nmean composite score  :", combined_gdf['mean_composite_score'].mean())
print("median composite score:", combined_gdf['median_composite_score'].mean())

# save to file
# combined_gdf.to_file((OUT_PATH / "routes_school1.gpkg"), driver="GPKG")

In [ ]:
combined_gdf.head(2)

### Loop all schools - by risk

In [ ]:
all_routes_gdf_by_risk, errors = compute_routes_from_census_blocks_to_all_schools(
    G,
    somerville_census_blocks=somerville_census_sample,
    schools_gdf=schools_gdf,
    weight="composite_score"
)

In [ ]:
# save only the combined file
all_routes_gdf_by_risk.to_file(OUT_PATH / "routes_by_risk.gpkg", driver="GPKG")

In [ ]:
all_routes_gdf_by_length, errors = compute_routes_from_census_blocks_to_all_schools(
    G,
    somerville_census_blocks=somerville_census_sample,
    schools_gdf=schools_gdf,
    weight="length"
)

In [ ]:
# save only the combined file
all_routes_gdf_by_length.to_file(OUT_PATH / "routes_by_length.gpkg", driver="GPKG")

### Summarize

Combine into one df, drop geometry

In [ ]:
all_routes_gdf_by_length['method'] = "length"
all_routes_gdf_by_risk['method'] = "risk"

all_routes_gdf = pd.concat([
    all_routes_gdf_by_length.drop(columns=['geometry']),
    all_routes_gdf_by_risk.drop(columns=['geometry'])
])

In [ ]:
school_means = all_routes_gdf.groupby(['school_name', 'method']).mean(numeric_only=True).reset_index()
school_means.head()

#### Sort by difference

- median_composite_score: $length - risk$

In [ ]:
# reset
school_means = all_routes_gdf.groupby(['school_name', 'method']).mean(numeric_only=True).reset_index()

# pivot to get length and risk methods as separate columns
pivoted = school_means.pivot(
    index='school_name', 
    columns='method', 
    values='median_composite_score'
).reset_index()

# calculate the difference (length - risk)
pivoted['score_diff_median'] = pivoted['length'] - pivoted['risk']

# merge back with the original data
school_means = school_means.merge(
    pivoted[['school_name', 'score_diff_median']], 
    on='school_name'
).sort_values('score_diff_median', ascending=False)

school_means.head()

In [ ]:
# Create the bar chart with sorting
chart = alt.Chart(school_means).mark_bar().encode(
    x=alt.X('school_name:N', 
            title='School', 
            axis=alt.Axis(labelAngle=-45),
            sort=alt.EncodingSortField(field='score_diff_median', order='descending')),
    y=alt.Y('mean_composite_score:Q', title='Mean Composite Score'),
    color=alt.Color('method:N', title='Method', scale=alt.Scale(scheme='set2')),
    xOffset='method:N',  # This creates grouped bars
    tooltip=[
        alt.Tooltip('school_name:N', title='School'),
        alt.Tooltip('method:N', title='Method'),
        alt.Tooltip('mean_composite_score:Q', title='Mean Score', format='.3f'),
        alt.Tooltip('sum_length:Q', title='Total Length', format='.1f')
    ]
).properties(
    width=600,
    height=400,
    title='Mean Composite Score by School and Method (Risk vs. Length)'
)

# Add text labels above each bar showing score_diff_median
text = alt.Chart(school_means).mark_text(
    align='center',
    baseline='bottom',
    dy=-5,  # Offset text above the bar
    fontSize=10
).encode(
    x=alt.X('school_name:N', sort=alt.EncodingSortField(field='score_diff_median', order='descending')),
    y=alt.Y('mean_composite_score:Q'),
    text=alt.Text('score_diff_median:Q', format='.3f'),
    xOffset='method:N'
)

# Combine the chart and text
final_chart = chart + text
final_chart

In [ ]:
block_means = all_routes_gdf.groupby(['from_block_geoid', 'method']).mean(numeric_only=True).reset_index()
block_means.head()

In [ ]:
block_group_means = all_routes_gdf.groupby(['from_blkgrp20', 'method']).mean(numeric_only=True).reset_index()
block_group_means.head()

In [ ]:
# Create the bar chart
chart = alt.Chart(block_group_means).mark_bar().encode(
    x=alt.X('from_blkgrp20:N', title='Block Group', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('mean_composite_score:Q', title='Mean Composite Score'),
    color=alt.Color('method:N', title='Method', scale=alt.Scale(scheme='set2')),
    xOffset='method:N',  # This creates grouped bars
    tooltip=[
        alt.Tooltip('school_name:N', title='School'),
        alt.Tooltip('method:N', title='Method'),
        alt.Tooltip('mean_composite_score:Q', title='Mean Score', format='.3f'),
        alt.Tooltip('sum_length:Q', title='Total Length', format='.1f')
    ]
).properties(
    width=600,
    height=400,
    title='Mean Composite Score by School and Method (Risk vs. Length)'
)

chart

### Loop all schools - by length